In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import reuters
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
(X_train, y_train), (X_test, y_test) = reuters.load_data(nb_words=10000, test_split=0.2)

In [ ]:
len(X_train)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import base_filter
from keras.preprocessing.text import text_to_word_sequence
tokenizer = Tokenizer(nb_words=10, filters=base_filter(), lower=True, split=" ")
texts = ['The cat sat on the mat.',
         'The dog sat on the log.',
         'Dogs and cats living together.']
tokenizer.fit_on_texts(texts)
sequences = []
for seq in tokenizer.texts_to_sequences_generator(texts):
    sequences.append(seq)

    
    
    

In [ ]:
tokenizer.nb_words

In [ ]:
from __future__ import print_function
from functools import reduce
import re
import tarfile

import numpy as np

from keras.datasets.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Merge
from keras.layers import recurrent
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences    
    
print('set vars')
RNN = recurrent.GRU
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 20

print('building sent rnn model')
sentrnn = Sequential()
sentrnn.add(Embedding(tokenizer.nb_words, EMBED_HIDDEN_SIZE, mask_zero=True))
sentrnn.add(RNN(SENT_HIDDEN_SIZE, return_sequences=False))

print('building qrnn')
qrnn = Sequential()
qrnn.add(Embedding(tokenizer.nb_words, EMBED_HIDDEN_SIZE))
qrnn.add(RNN(QUERY_HIDDEN_SIZE, return_sequences=False))

model = Sequential()
model.add(Merge([sentrnn, qrnn], mode='concat'))
model.add(Dense(tokenizer.nb_words, activation='softmax'))

print('compiling model')
model.compile(optimizer='adam', loss='categorical_crossentropy', class_mode='categorical')

print('Training')
#model.fit([X, Xq], Y, batch_size=BATCH_SIZE, nb_epoch=EPOCHS, validation_split=0.05, show_accuracy=True)

In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import reuters
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer

max_words = 1000
batch_size = 32
nb_epoch = 5

print('Loading data...')
(X_train, y_train), (X_test, y_test) = reuters.load_data(nb_words=max_words, test_split=0.2)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

nb_classes = np.max(y_train)+1
print(nb_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(nb_words=max_words)
X_train = tokenizer.sequences_to_matrix(X_train, mode='binary')
X_test = tokenizer.sequences_to_matrix(X_test, mode='binary')
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Convert class vector to binary class matrix (for use with categorical_crossentropy)')
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
print('Y_train shape:', Y_train.shape)
print('Y_test shape:', Y_test.shape)

print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

history = model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1, show_accuracy=True, validation_split=0.1)
score = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=1, show_accuracy=True)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from keras.datasets import imdb
max_features = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features, test_split=0.2)

In [ ]:
from keras.preprocessing import sequence
X_train_pad = sequence.pad_sequences(X_train, maxlen=500)
X_test_pad = sequence.pad_sequences(X_test, maxlen=500)


In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Merge
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.preprocessing.text import Tokenizer
from keras.regularizers import l1
print "building graph..."
query_model = Sequential()
#query_model.add(Embedding(num_query_features, 128))
query_model.add(Embedding(10000, 128, input_length=10))
query_model.add(LSTM(128))
query_model.add(Dropout(0.5))


text_model = Sequential()
text_model.add(Embedding(10000, 128, input_length=100))
text_model.add(LSTM(128))
text_model.add(Dropout(0.5))

combined_model = Sequential()
combined_model.add(Merge([query_model, text_model], mode='concat'))
combined_model.add(Dense(64, input_shape=(256,), activation='relu'))
combined_model.add(Dense(1, input_shape=(100,), activation='sigmoid'))
combined_model.compile(loss='binary_crossentropy', optimizer='adam', class_mode="binary")


In [ ]:
1500 / 60

In [ ]:
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in xrange(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

In [ ]:
import theano
theano.test()

In [1]:
0.26 ** 0.5

0.5099019513592785